In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X = df.drop('bought_insurance', axis = 1)
y = df.iloc[:, -1]

X['age'] = X['age'] / 100

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [4]:
# defining the sigmoid function

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
# calculating cost using log-loss

def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred_new = [max(i, epsilon) for i in y_pred]
    y_pred_new = [min(i, 1 - epsilon) for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    
    return -np.mean(y_true * np.log(y_pred_new) + (1-y_true) * np.log(1-y_pred_new))

In [6]:
# creating custom neural network 

class NN():
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_threshold)

    def predict(self, X):
        weighted_sum = self.w1 * X['age'] + self.w2 * X['affordibility'] + self.bias
        return sigmoid(weighted_sum)

    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
        w1 = 1
        w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
    
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordibility + bias
            y_pred = sigmoid(weighted_sum)
            loss = log_loss(y_true, y_pred)
        
            w1_derivative = (1/n) * np.dot(np.transpose(age), (y_pred - y_true))
            w2_derivative = (1/n) * np.dot(np.transpose(affordibility), (y_pred - y_true))
            bias_derivative = np.mean(y_pred - y_true)
        
            w1 = w1 - rate * w1_derivative
            w2 = w2 - rate * w2_derivative
            bias = bias - rate * bias_derivative
            
            if (i+1)%50 == 0:
                print(f'Epoch : {i+1}, w1 : {round(w1, 4)}, w2 : {round(w2, 4)}, bias = {round(bias, 4)}, loss : {round(loss, 4)}')
    
            if loss <= loss_threshold:
                print(f'Epoch : {i+1}, w1 : {round(w1, 4)}, w2 : {round(w2, 4)}, bias = {round(bias, 4)}, loss : {round(loss, 4)}')
                break
                
        return w1, w2, bias

In [7]:
model = NN()
model.fit(X_train, y_train, epochs = 500, loss_threshold = 0.45)

Epoch : 50, w1 : 1.4254, w2 : 1.1244, bias = -1.3008, loss : 0.5677
Epoch : 100, w1 : 2.0639, w2 : 1.395, bias = -1.7924, loss : 0.5384
Epoch : 150, w1 : 2.6656, w2 : 1.5325, bias = -2.1576, loss : 0.5177
Epoch : 200, w1 : 3.2235, w2 : 1.6071, bias = -2.4509, loss : 0.5015
Epoch : 250, w1 : 3.7371, w2 : 1.6506, bias = -2.699, loss : 0.4883
Epoch : 300, w1 : 4.2084, w2 : 1.6784, bias = -2.916, loss : 0.4775
Epoch : 350, w1 : 4.6406, w2 : 1.6986, bias = -3.1102, loss : 0.4685
Epoch : 400, w1 : 5.037, w2 : 1.715, bias = -3.2864, loss : 0.4609
Epoch : 450, w1 : 5.4011, w2 : 1.7298, bias = -3.4478, loss : 0.4545
Epoch : 492, w1 : 5.6844, w2 : 1.7416, bias = -3.5738, loss : 0.4499


In [8]:
model.predict(X_test)

7     0.459303
21    0.109502
5     0.794312
2     0.698364
13    0.127271
19    0.308115
dtype: float64